In [1]:
%pip install scipy torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 133.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 198.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 210.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 kB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 35.4 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pi

In [3]:
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 135.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 161.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 199.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 175.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
import os
import scipy.io
import numpy as np
import torch
import torchvision
from torchsummary import summary
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.models import MobileNet_V3_Large_Weights
from torch.optim import Adam
from torch import nn


## Load Dataset

**Define Custom Dataset**

In [56]:
from PIL import Image
from torch.utils.data import Dataset
import os
import torch
import scipy.io

class OxfordFlowersDataset(Dataset):
    def __init__(self, image_folder, label_file, transform=None):
        """
        :param image_folder: Path to the local folder containing images.
        :param label_file: Path to the .mat file containing labels.
        :param transform: Transformations to apply to the images.
        """
        self.image_folder = image_folder
        self.label_file = label_file
        self.transform = transform
        self.image_files = self._get_image_files()
        self.labels = self._load_labels()

    def _get_image_files(self):
        """Retrieve all image file names from the local folder."""
        image_files = [f for f in os.listdir(self.image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        return sorted(image_files)  # Ensure images are loaded in order

    def _load_labels(self):
        """Load labels from a local .mat file."""
        label_data = scipy.io.loadmat(self.label_file)
        image_labels = label_data['labels'].flatten() - 1  # Convert 1-based to 0-based indexing
        return torch.tensor(image_labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        file_name = self.image_files[idx]
        image_path = os.path.join(self.image_folder, file_name)
        label = self.labels[idx]

        try:
            # Open image using PIL
            image = Image.open(image_path).convert("RGB")

        except Exception as e:
            print(f"Error reading image {file_name}: {e}")
            return None, None

        # Apply transformations if specified
        if self.transform:
            image = self.transform(image)

        return image, label

## Training Prep

Proportion:
- 80:20 - train:val
- 256 batch size
- GPU enabled on RunPod

Transformation:
- resize to 224x224
- horizonal flip
- convert to tensor
- normalize image

In [57]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = OxfordFlowersDataset(
    image_folder = "images/jpg", 
    label_file = "imagelabels.mat",
    transform   = transform
)

# Split dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=2
)

val_loader = DataLoader(
    val_dataset, batch_size=256, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=2
)



In [58]:
print(f"Number of images: {len(dataset.image_files)}")
print(f"Number of labels: {len(dataset.labels)}")

Number of images: 8189
Number of labels: 8189


In [59]:
# Load MobileNetV3 model
model = models.mobilenet_v3_large(pretrained=True)

# Modify the classifier BEFORE moving the model to GPU
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 102)  # Adjust to 102 classes

# Define device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Move model to the correct device
model = model.to(device)

# Check if the model is on GPU
for param in model.parameters():
    assert param.device == device, f"Model parameter is on {param.device}, not {device}!"

print("Model successfully moved to:", device)

Model successfully moved to: cuda:0


In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

## Model Training

In [61]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    """Train the model and return losses, accuracies, and elapsed time."""
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move to device

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100. * correct / total
        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_acc)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_acc = 100. * correct / total
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, "
              f"Train Acc: {train_accuracies[-1]:.2f}%, Val Loss: {val_losses[-1]:.4f}, "
              f"Val Acc: {val_accuracies[-1]:.2f}%")

    end_time = time.time()
    elapsed_time = end_time - start_time

    return model, train_losses, val_losses, train_accuracies, val_accuracies, elapsed_time

In [62]:
def plot_and_save_curves(train_losses, val_losses, train_accuracies, val_accuracies):
    """Plot loss and accuracy curves and save them as images."""
    
    # Plot losses
    plt.figure()
    plt.plot(train_losses, label='Training Loss', marker='o')
    plt.plot(val_losses, label='Validation Loss', marker='s')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Learning Curve: Loss')
    plt.savefig("loss_curve.png")  # Save loss curve
    plt.close()

    # Plot accuracies
    plt.figure()
    plt.plot(train_accuracies, label='Training Accuracy', marker='o')
    plt.plot(val_accuracies, label='Validation Accuracy', marker='s')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.title('Learning Curve: Accuracy')
    plt.savefig("accuracy_curve.png")  # Save accuracy curve
    plt.close()

    print("Saved plots as 'loss_curve.png' and 'accuracy_curve.png'")

In [63]:
# Train the model
model, train_losses, val_losses, train_accuracies, val_accuracies, elapsed_time = train_model(
    model, criterion, optimizer, train_loader, val_loader, num_epochs=10
)

print(f"\nElapsed time: {elapsed_time:.2f} seconds\n")

# Plot and save learning curves
plot_and_save_curves(train_losses, val_losses, train_accuracies, val_accuracies)

Epoch 1/10, Train Loss: 1.5461, Train Acc: 66.34%, Val Loss: 3.6893, Val Acc: 29.91%
Epoch 2/10, Train Loss: 0.1258, Train Acc: 96.57%, Val Loss: 1.6793, Val Acc: 66.00%
Epoch 3/10, Train Loss: 0.0428, Train Acc: 98.93%, Val Loss: 0.8085, Val Acc: 80.77%
Epoch 4/10, Train Loss: 0.0155, Train Acc: 99.56%, Val Loss: 0.4608, Val Acc: 88.40%
Epoch 5/10, Train Loss: 0.0070, Train Acc: 99.86%, Val Loss: 0.3212, Val Acc: 92.00%
Epoch 6/10, Train Loss: 0.0069, Train Acc: 99.83%, Val Loss: 0.1840, Val Acc: 95.30%
Epoch 7/10, Train Loss: 0.0123, Train Acc: 99.65%, Val Loss: 0.3267, Val Acc: 92.31%
Epoch 8/10, Train Loss: 0.0099, Train Acc: 99.82%, Val Loss: 0.2077, Val Acc: 95.24%
Epoch 9/10, Train Loss: 0.0137, Train Acc: 99.66%, Val Loss: 0.2155, Val Acc: 94.99%
Epoch 10/10, Train Loss: 0.0134, Train Acc: 99.51%, Val Loss: 0.3145, Val Acc: 93.10%

Elapsed time: 121.34 seconds

Saved plots as 'loss_curve.png' and 'accuracy_curve.png'
